# Double Demeaning for Fixed Effects Interactions - Basic Example

This notebook demonstrates the use of the `dd_ie` package for analyzing interactions in fixed effects models using the double demeaning technique.

**Reference:** Giesselmann, M., & Schmidt-Catran, A. W. (2022). Interactions in Fixed Effects Regression Models. *Sociological Methods & Research*, 51(3), 1100-1127.

**Author:** Nikolaos Koutounidis


## Setup and Imports

First, let's import the necessary libraries and add the current directory to the Python path so we can import our modules directly.


In [1]:
import sys
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Get the parent directory (dd_ie directory)
notebook_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))
parent_dir = os.path.dirname(notebook_dir)

# Add both directories to path
for path in [parent_dir, notebook_dir]:
    if path not in sys.path:
        sys.path.insert(0, path)

print(f"📂 Notebook directory: {notebook_dir}")
print(f"📂 Parent directory: {parent_dir}")

# Import strategy with multiple fallbacks
import_success = False

# Strategy 1: Try importing as installed package
try:
    from dd_ie import DoubleDemeanAnalysis
    print("✅ Strategy 1: Successfully imported dd_ie as installed package")
    import_success = True
except ImportError:
    print("❌ Strategy 1: dd_ie package not found")

# Strategy 2: Try importing the modules directly with package simulation
if not import_success:
    try:
        # Manually set up the package structure
        import importlib.util
        import types
        
        # Create a module object for dd_ie
        dd_ie = types.ModuleType('dd_ie')
        sys.modules['dd_ie'] = dd_ie
        
        # Load utils first (no dependencies)
        utils_path = os.path.join(parent_dir, 'utils.py')
        spec = importlib.util.spec_from_file_location("dd_ie.utils", utils_path)
        utils_module = importlib.util.module_from_spec(spec)
        sys.modules['dd_ie.utils'] = utils_module
        spec.loader.exec_module(utils_module)
        
        # Now load core (depends on utils)
        core_path = os.path.join(parent_dir, 'core.py')
        spec = importlib.util.spec_from_file_location("dd_ie.core", core_path)
        core_module = importlib.util.module_from_spec(spec)
        sys.modules['dd_ie.core'] = core_module
        spec.loader.exec_module(core_module)
        
        # Import what we need
        DoubleDemeanAnalysis = core_module.DoubleDemeanAnalysis
        print("✅ Strategy 2: Successfully imported modules with package simulation")
        import_success = True
        
    except Exception as e:
        print(f"❌ Strategy 2: Package simulation failed - {e}")

# Strategy 3: Manual import with modified relative imports
if not import_success:
    try:
        # Change to parent directory temporarily
        original_dir = os.getcwd()
        os.chdir(parent_dir)
        
        # Import utils first
        exec(open('utils.py').read())
        
        # Read core.py and modify the relative import
        with open('core.py', 'r') as f:
            core_code = f.read()
        
        # Replace relative import with direct import
        core_code = core_code.replace('from .utils import', 'from utils import')
        
        # Execute the modified code
        exec(core_code)
        
        # Change back to original directory
        os.chdir(original_dir)
        
        print("✅ Strategy 3: Successfully imported with manual code execution")
        import_success = True
        
    except Exception as e:
        print(f"❌ Strategy 3: Manual import failed - {e}")
        os.chdir(original_dir)  # Make sure we change back

if not import_success:
    print("❌ All import strategies failed!")
    print("Please try installing the package first with: pip install -e .")
    raise ImportError("Could not import dd_ie modules")

print(f"\n📦 Using pandas version: {pd.__version__}")
print(f"📦 Using numpy version: {np.__version__}")
print("🎉 Ready to proceed with analysis!")

📂 Notebook directory: c:\Users\nkoutoun\OneDrive - UGent\Desktop\Research\Projects Active\COVID-19 Project\covid_paper
📂 Parent directory: c:\Users\nkoutoun\OneDrive - UGent\Desktop\Research\Projects Active\COVID-19 Project
✅ Strategy 1: Successfully imported dd_ie as installed package

📦 Using pandas version: 2.2.2
📦 Using numpy version: 1.26.4
🎉 Ready to proceed with analysis!


## Generate Synthetic Panel Data

Let's create a realistic panel dataset that demonstrates the potential for bias in standard fixed effects interaction estimates. This data includes:

- **Unit-specific heterogeneity**: Each unit has its own interaction effect
- **Within-unit variation**: Both X and Z vary over time within units
- **Realistic structure**: 50 units × 8 time periods


In [2]:
def generate_synthetic_data(n_units=50, n_time=8, seed=42):
    """
    Generate synthetic panel data for demonstration.
    
    This creates a realistic panel dataset where:
    - Both X and Z vary within units
    - There's potential for unobserved effect heterogeneity
    - The true interaction effect differs from standard FE estimates
    """
    np.random.seed(seed)
    
    data = []
    
    for unit in range(1, n_units + 1):
        # Unit-specific parameters (unobserved heterogeneity)
        alpha_i = np.random.normal(0, 1)  # Unit fixed effect
        beta_interaction_i = np.random.normal(0.5, 0.2)  # Unit-specific interaction effect
        
        for time in range(1, n_time + 1):
            # Time-varying variables
            x_it = np.random.normal(unit/10, 1)  # Treatment variable with unit-specific trend
            z_it = np.random.normal(time/5, 1)   # Moderator variable with time trend
            control_it = np.random.normal(0, 1)  # Control variable
            
            # Generate outcome with unit-specific interaction effect
            # This creates the bias that double demeaning corrects for
            y_it = (alpha_i + 
                   0.8 * x_it + 
                   0.6 * z_it + 
                   beta_interaction_i * x_it * z_it +  # Unit-specific interaction
                   0.3 * control_it + 
                   np.random.normal(0, 0.5))  # Random error
            
            data.append({
                'unit_id': unit,
                'time_id': time,
                'outcome': y_it,
                'treatment': x_it,
                'moderator': z_it,
                'control': control_it
            })
    
    return pd.DataFrame(data)

# Generate the data
print("📊 Generating synthetic panel data...")
df = generate_synthetic_data(n_units=50, n_time=8)

print(f"✅ Created dataset with {len(df):,} observations")
print(f"   📈 Units: {df['unit_id'].nunique()}")
print(f"   📅 Time periods: {df['time_id'].nunique()}")
print(f"   📊 Variables: {list(df.columns)}")


📊 Generating synthetic panel data...
✅ Created dataset with 400 observations
   📈 Units: 50
   📅 Time periods: 8
   📊 Variables: ['unit_id', 'time_id', 'outcome', 'treatment', 'moderator', 'control']


## Explore the Data

Let's take a look at the structure and properties of our synthetic dataset.


In [3]:
# Display first few rows
print("📋 First 10 rows of the dataset:")
display(df.head(10))

# Summary statistics
print("\n📊 Summary statistics:")
display(df.describe())


📋 First 10 rows of the dataset:


,unit_id,time_id,outcome,treatment,moderator,control
0,1,1,2.549888,0.747689,1.723030,-0.234153
1,1,2,3.596959,1.679213,1.167435,-0.469474
2,1,3,-0.620558,-0.363418,0.134270,0.241962
3,1,4,-0.989769,-1.624918,0.237712,-1.012831
4,1,5,0.087082,-0.808024,-0.412304,1.465649
5,1,6,0.370250,0.167528,-0.224748,-0.544383
6,1,7,-0.486217,-1.050994,1.775698,-0.600639
7,1,8,0.815691,-0.501707,3.452278,-0.013497
8,2,1,-0.390253,0.408864,-1.759670,-1.328186
9,2,2,1.868072,0.938467,0.571368,-0.115648



📊 Summary statistics:


,unit_id,time_id,outcome,treatment,moderator,control
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,25.500000,4.500000,3.616810,2.557174,0.859166,0.106163
std,14.448942,2.294157,3.433580,1.753068,1.090468,0.960648
min,1.000000,1.000000,-3.386093,-2.319745,-2.448543,-3.241267
25%,13.000000,2.750000,1.186356,1.249178,0.089275,-0.546654
50%,25.500000,4.500000,3.248646,2.612533,0.791704,0.101767
75%,38.000000,6.250000,5.615207,3.916344,1.604203,0.759596
max,50.000000,8.000000,23.232784,6.962525,4.055300,2.573360


## Initialize Double Demeaning Analysis

Now let's set up the analysis using the `DoubleDemeanAnalysis` class. This will validate our data and prepare it for the double demeaning procedure.


In [4]:
print("🔬 Initializing Double Demeaning Analysis...")
print("=" * 60)

analysis = DoubleDemeanAnalysis(
    data=df,
    unit_var='unit_id',
    time_var='time_id', 
    y_var='outcome',
    x_var='treatment',
    z_var='moderator',
    w_vars=['control']
)

print("✅ Analysis object created successfully!")
print(f"   🎯 Dependent variable: {analysis.y_var}")
print(f"   ⚡ Interaction: {analysis.x_var} × {analysis.z_var}")
print(f"   🎛️ Controls: {analysis.w_vars}")


🔬 Initializing Double Demeaning Analysis...
📊 Panel Data Summary:
   Total observations: 400
   Number of units: 50
   Time periods per unit: 8-8
   Panel type: Balanced
✅ Data prepared: 400 observations, 50 units
✅ Analysis object created successfully!
   🎯 Dependent variable: outcome
   ⚡ Interaction: treatment × moderator
   🎛️ Controls: ['control']


## Run the Complete Analysis

This is the main analysis step. The `run_analysis()` method will:

1. **Data preparation**: Validate panel structure and check variation
2. **Grand mean centering**: Optional preprocessing step
3. **Double demeaning**: Create the unbiased interaction term
4. **Model estimation**: Compare standard FE vs double demeaned FE
5. **Statistical testing**: Hausman test for systematic differences


In [5]:
# Run the complete analysis with verbose output
results = analysis.run_analysis(
    center_variables=True,  # Apply grand mean centering
    run_hausman=True,      # Perform Hausman test
    verbose=True           # Show detailed output
)


🚀 STARTING DOUBLE DEMEANING ANALYSIS
Dataset: 400 observations, 4 variables
Panel structure: unit_id (units) × time_id (time)
Analysis: outcome ~ treatment × moderator + controls

📋 Step 1: Data Preparation
   Panel info: 1 units, 8-8 periods per unit
   Average periods per unit: 8.0

📋 Step 2: Grand Mean Centering
  outcome: mean before = 3.61681, mean after = 0.0000000000
  treatment: mean before = 2.55717, mean after = -0.0000000000
  moderator: mean before = 0.85917, mean after = 0.0000000000
  control: mean before = 0.10616, mean after = -0.0000000000

📋 Step 3: Double Demeaning Implementation
CREATING DOUBLE DEMEANED INTERACTION
Step 1: Computing within-unit means...
  treatment: unit means calculated -> mean_treatment
    Example: Unit 1 mean = -2.77650
  moderator: unit means calculated -> mean_moderator
    Example: Unit 1 mean = 0.12251

Step 2: Creating within-unit demeaned variables...
  treatment -> dm_treatment = treatment - mean_treatment
    Mean of demeaned var: 0.0000

## Interpret the Results

Now let's extract and interpret the key findings from our analysis.


In [6]:
print("=" * 80)
print("📋 RESULTS SUMMARY")
print("=" * 80)

# Extract key results
standard_results = results['standard_results']
dd_results = results['dd_results']
comparison_df = results['comparison_df']
hausman_test = results['hausman_test']

print(f"📊 Available result components:")
for key in results.keys():
    print(f"   📋 {key}")

print(f"\n🎯 Analysis completed successfully!")

# Show coefficient comparison for the interaction effect
print("\n🔍 INTERACTION EFFECT COMPARISON:")
print("=" * 50)

if len(comparison_df) > 0:
    # Find the interaction row
    interaction_row = comparison_df[comparison_df['Variable'].str.contains('int_')]
    
    if len(interaction_row) > 0:
        row = interaction_row.iloc[0]
        std_coef = row['Std FE Coef']
        dd_coef = row['DD Coef']
        difference = row['Difference']
        
        print(f"📈 Standard FE interaction coefficient: {std_coef:.6f}")
        print(f"📈 Double Demeaned interaction coefficient: {dd_coef:.6f}")
        print(f"📊 Difference: {difference:.6f}")
        
        # Interpretation
        if abs(difference) > 0.1:
            print(f"\n⚠️  SUBSTANTIAL DIFFERENCE DETECTED!")
            print(f"   → Standard FE interaction may be biased")
            print(f"   → Double demeaned estimator is preferred")
        else:
            print(f"\n✅ Small difference between estimators")
            print(f"   → Both estimators appear similar")
    
    print(f"\n📋 Full coefficient comparison table:")
    display(comparison_df)
else:
    print("No comparison data available")


📋 RESULTS SUMMARY
📊 Available result components:
   📋 standard_results
   📋 dd_results
   📋 comparison_df
   📋 hausman_test
   📋 processed_data

🎯 Analysis completed successfully!

🔍 INTERACTION EFFECT COMPARISON:

📋 Full coefficient comparison table:


,Variable,Std FE Coef,Std FE SE,DD Coef,DD SE,Difference
0,treatment,1.229217,0.049972,1.225278,0.074700,0.003939
1,control,0.322545,0.042245,0.326583,0.058093,-0.004039
2,moderator,1.832722,0.071099,1.767363,0.143131,0.065359


In [7]:
# Hausman test interpretation
print("🔬 HAUSMAN TEST RESULTS:")
print("=" * 40)

if hausman_test:
    test_stat = hausman_test['hausman_statistic']
    p_value = hausman_test['p_value']
    df_test = hausman_test['degrees_of_freedom']
    conclusion = hausman_test['conclusion']
    
    print(f"📊 Test statistic (χ²): {test_stat:.4f}")
    print(f"📊 Degrees of freedom: {df_test}")
    print(f"📊 P-value: {p_value:.4f}")
    print(f"📋 Conclusion: {conclusion}")
    
    print(f"\n💡 INTERPRETATION:")
    if p_value < 0.05:
        print(f"   🚨 REJECT null hypothesis (p = {p_value:.4f} < 0.05)")
        print(f"   → Evidence of systematic differences between estimators")
        print(f"   → Standard FE may be inconsistent - prefer double demeaned estimator")
        print(f"   → There is likely unobserved effect heterogeneity")
    else:
        print(f"   ✅ FAIL TO REJECT null hypothesis (p = {p_value:.4f} ≥ 0.05)")
        print(f"   → No systematic differences detected")
        print(f"   → Both estimators appear consistent; standard FE is more efficient")
        print(f"   → No strong evidence of unobserved effect heterogeneity")
        
    if not hausman_test.get('positive_definite', True):
        print(f"\n📝 NOTE: Variance matrix was not positive definite")
        print(f"   Test statistic computed using generalized inverse")
        print(f"   This can occur with small samples or high collinearity")
else:
    print("❌ Hausman test results not available")


🔬 HAUSMAN TEST RESULTS:
📊 Test statistic (χ²): 0.5128
📊 Degrees of freedom: 4
📊 P-value: 0.9722
📋 Conclusion: NO SYSTEMATIC BIAS

💡 INTERPRETATION:
   ✅ FAIL TO REJECT null hypothesis (p = 0.9722 ≥ 0.05)
   → No systematic differences detected
   → Both estimators appear consistent; standard FE is more efficient
   → No strong evidence of unobserved effect heterogeneity


## Conclusion

This notebook demonstrated the complete workflow for double demeaning analysis:

1. **✅ Data Preparation**: Generated synthetic panel data with realistic structure
2. **✅ Analysis Setup**: Initialized the DoubleDemeanAnalysis class
3. **✅ Model Estimation**: Compared standard FE vs double demeaned FE models
4. **✅ Statistical Testing**: Performed Hausman test for systematic differences
5. **✅ Results Interpretation**: Provided clear guidance on which estimator to prefer

### Key Takeaways:

- **Double demeaning** provides an unbiased within-unit interaction estimator when there is unobserved effect heterogeneity
- **Hausman test** helps determine whether standard FE estimates are biased
- **Both approaches** should be reported for transparency and robustness

### For Your Research:

1. Replace the synthetic data with your real dataset
2. Adjust variable names to match your data
3. Interpret results in the context of your research question
4. Consider theoretical justifications for the interaction effects

**Happy analyzing! 📊🔬**
